In [40]:
import sympy as sp
from sympy import cos,sin,Matrix
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
import numpy as np
from scipy.integrate import solve_ivp
init_vprinting()

In [66]:
(m1,m2,m,l,l1,l2,l3,t) = sp.symbols("m_1,m_2,m,l,l_1,l_2,l_3,t")
(theta1,theta2,phi) = dynamicsymbols("theta_1,theta_2,phi")
dtheta1 = sp.diff(theta1)
dtheta2 = sp.diff(theta2)
dphi = sp.diff(phi)

ddtheta1 = sp.diff(dtheta1)
ddtheta2 = sp.diff(dtheta2)
ddphi = sp.diff(dphi)

q = sp.Matrix([theta1,theta2,phi]);
dq = sp.Matrix([dtheta1,dtheta2,dphi]);
ddq = sp.Matrix([ddtheta1,ddtheta2,ddphi]);


l1 = l
l2 = l
l3 = l

m1 = m
m2 = m
m3 = m

In [97]:
I1 = m1*l1**2
I2 = m2*l2**2
I3 = m1*(1/4*l3**2+l1**2-2*l3*l1*sp.cos(theta1))+m2*(1/4*l3**2+l2**2-2*l3*l2*sp.cos(theta2))

In [88]:
K1 = 1/2*I1*dtheta1**2
K2 = 1/2*I2*dtheta2**2
K3 = 1/2*I3*dphi**2

In [89]:
L1 = K1
L2 = K2
L3 = K3

In [100]:
sp.expand(L3)

In [91]:
EOM_1 = sp.diff(sp.diff(L1,dq[0])+sp.diff(L1,dq[1])+sp.diff(L1,dq[2]),t)-sp.diff(L1,q[0])-sp.diff(L1,q[1])-sp.diff(L1,q[2])
EOM_2 = sp.diff(sp.diff(L2,dq[0])+sp.diff(L2,dq[1])+sp.diff(L2,dq[2]),t)-sp.diff(L2,q[0])-sp.diff(L2,q[1])-sp.diff(L2,q[2])
EOM_3 = sp.diff(sp.diff(L3,dq[0])+sp.diff(L3,dq[1])+sp.diff(L3,dq[2]),t)-sp.diff(L3,q[0])-sp.diff(L3,q[1])-sp.diff(L3,q[2])


In [92]:
A = sp.zeros(3,2)
A[2,0] = sp.simplify(EOM_3.expand().subs([[ddtheta1,0],[ddtheta2,0],[ddphi,0]]).coeff(dtheta1)/dphi/m/l**2)
A[2,1] = sp.simplify(EOM_3.expand().subs([[ddtheta1,0],[ddtheta2,0],[ddphi,0]]).coeff(dtheta2)/dphi/m/l**2)
D = sp.simplify(EOM_3.expand().subs([[ddtheta1,0],[ddtheta2,0],[ddphi,0]]).coeff(dphi**2)/m/l**2)

In [93]:
A

⎡     0            0     ⎤
⎢                        ⎥
⎢     0            0     ⎥
⎢                        ⎥
⎣2.0⋅sin(θ₁)  2.0⋅sin(θ₂)⎦

In [26]:
EOM_3

In [22]:
EOM_3.expand()

In [151]:
M @ ddq + C @ dq 

⎡                                                                          2  
⎢                                                                     1.0⋅l ⋅m
⎢                                                                             
⎢                                                                          2  
⎢                                                                     1.0⋅l ⋅m
⎢                                                                             
⎢⎛     2                           2               2                          
⎣⎝1.0⋅l ⋅m₁⋅sin(θ₁)⋅cos(θ₁) + 0.5⋅l ⋅m₁⋅sin(θ₁) - l ⋅m₂⋅sin(θ₂)⋅cos(θ₂) - 0.5⋅

       ⎛       2                         2             ⎞                      
₁⋅θ₁̈ + ⎝- 2.0⋅l ⋅m₁⋅sin(θ₁)⋅cos(θ₁)⋅φ̇ - l ⋅m₁⋅sin(θ₁)⋅φ̇⎠⋅θ₁̇               
                                                                              
       ⎛       2                         2             ⎞                      
₂⋅θ₂̈ + ⎝- 2.0⋅l ⋅m₂⋅sin(θ₂)⋅cos(θ₂)⋅φ̇ - l ⋅m₂⋅sin

In [93]:
M = sp.Matrix([[0.5*l**2*m1,0,0],[0,0.5*m2*l**2,0],[0,0,0.125*m1*l**2*(2*sp.cos(theta1)+1)**2+0.125*m2*l**2*(2*sp.cos(theta2)+1)**2]])

In [92]:
M

⎡     2                                                                       
⎢0.5⋅l ⋅m₁     0                                  0                           
⎢                                                                             
⎢                 2                                                           
⎢    0      0.5⋅m₂                                0                           
⎢                                                                             
⎢                           2                   2          2                  
⎣    0         0     0.125⋅l ⋅m₁⋅(2⋅cos(θ₁) + 1)  + 0.125⋅l ⋅m₂⋅(2⋅cos(θ₂) + 1

  ⎤
  ⎥
  ⎥
  ⎥
  ⎥
  ⎥
 2⎥
) ⎦

In [109]:
C = sp.Matrix([[m1*cos(theta1 + 0.5)*sin(theta1)*dphi,0,0],[0,m2*cos(theta1 + 0.5)*sin(theta1)*dphi,0],[0,0,0.5*l*m1*(l*cos(theta2) + 0.5*l)*sin(theta2)*dphi+0.5*l*m1*(l*cos(theta1) + 0.5*l)*sin(theta1)*dphi]])

In [110]:
C

[0.5⋅l⋅m₁⋅(l⋅cos(θ₁) + 0.5⋅l)⋅sin(θ₁)⋅φ̇ + 0.5⋅l⋅m₁⋅(l⋅cos(θ₂) + 0.5⋅l)⋅sin(θ₂
)⋅φ̇]

In [117]:
(m1*(cos(theta1 + 0.5)*sin(theta1)*dtheta1 + m2*(cos(theta2) + 0.5)*sin(theta_2)*diff(theta2, t))*Derivative(phi(t), t))

NameError: name 'm_2' is not defined

# Part 2

In [35]:
alpha1,alpha2 = sp.symbols("alpha_1,alpha_2")
D = 9 + 6*(cos( alpha1 ) + cos( alpha2 )) +2*cos ( alpha1 - alpha2 )
A = -1/D*sp.Matrix([[0,0],[0,0],[5 + 3*cos( alpha2 ) + cos( alpha1 - alpha2 ),5 + 3*cos ( alpha1 ) + cos ( alpha1 - alpha2 )]]) 

nA = sp.lambdify([alpha1,alpha2],A)

def dynamics(t,)